In [ ]:
import openai
import requests
from flask import Flask, request, jsonify
from transformers import pipeline

In [ ]:
# Initialize Flask app
app = Flask(__name__)

In [ ]:
# Set up OpenAI API (for GPT-3/4)
openai.api_key = 'OpenAI_API_KEY'

In [ ]:
# Initialize Hugging Face GPT model for fallback
#chatbot = pipeline("text-generation", model="microsoft/DialoGPT-medium")

In [ ]:
# Weather API key (OpenWeatherMap)
weather_api_key = 'WEATHER_API_KEY'

In [ ]:
# Flight API Key (Amadeus)
amadeus_client_id = 'Amadeus_API'
amadeus_client_secret = 'Secret'

In [ ]:
# Flight API Key
flight_api_key ="Amadeus_API"

In [ ]:
# Route for chatbot interaction
@app.route("/chat", methods=["POST"])
def chat():
    user_message = request.json.get('message')
# Check if the message contains a travel-specific request like weather or flight info
    if "weather" in user_message.lower():
        return get_weather_info(user_message)
    elif "flight" in user_message.lower():
        return get_flight_info(user_message)
    else:
        # Default to GPT-3/4 for general conversation
        return get_gpt_response(user_message)

In [ ]:
# Get response from OpenAI GPT-3/4 (or GPT-4 via OpenAI API)
def get_gpt_response(message):
    response = openai.Completion.create(
        engine="text-davinci-003",  
        prompt=message,
        max_tokens=100,
        temperature=0.7,
    )
    return jsonify({"response": response.choices[0].text.strip()})

In [ ]:
# Fetch weather data from OpenWeatherMap API
def get_weather_info(message):
    location = extract_location(message)
    if location:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
        response = requests.get(url).json()
        if response.get("cod") == 200:
            weather_data = response["main"]
            weather_description = response["weather"][0]["description"]
            return jsonify({"response": f"The weather in {location} is {weather_description} with a temperature of {weather_data['temp']}°C."})
        else:
            return jsonify({"response": "Sorry, I couldn't fetch the weather data for that location."})
    else:
        return jsonify({"response": "Please provide a location for the weather."})

In [ ]:
# Extract location from the user's message (simplified version)
def extract_location(message):
    # For simplicity, we assume the location is the last word in the sentence (could be extended)
    words = message.split()
    return words[-1] if words else None

In [ ]:
# Fetch flight information using Amadeus API
def get_flight_info(message):
    # For simplicity, let's extract departure and destination (real code would be more complex)
    if "from" in message and "to" in message:
        parts = message.split("from")
        from_location = parts[1].split("to")[0].strip()
        to_location = parts[1].split("to")[1].strip()

        # Fetch flight data from Amadeus API
        flight_data = search_flights(from_location, to_location)
        
        if flight_data:
            return jsonify({"response": f"Found flights from {from_location} to {to_location}. Details: {flight_data}"})
        else:
            return jsonify({"response": f"Sorry, I couldn't find any flights from {from_location} to {to_location}."})
    else:
        return jsonify({"response": "Please provide both the departure and destination locations."})

In [ ]:
# Fetch flight information from Amadeus API
def search_flights(from_location, to_location):
    token = get_amadeus_access_token()
    if token:
        url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
        headers = {
            "Authorization": f"Bearer {token}"
        }
        params = {
            "originLocationCode": from_location.upper(), 
            "destinationLocationCode": to_location.upper(),  
            "departureDate": "2025-05-10", 
            "adults": 1
        }
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            flight_data = response.json()
            # Simplify the data for response (you could improve this based on your needs)
            return flight_data.get("data", [])
        else:
            print(f"Error: {response.status_code}, {response.text}")
            return None
    else:
        print("Unable to get access token.")
        return None

In [ ]:
def get_amadeus_access_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": amadeus_client_id,
        "client_secret": amadeus_client_secret
    }
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

if __name__ == '__main__':
    app.run(debug=True)